# Ingest CXG spatial h5ad file and create SOMA object on disk

In [1]:
import tiledbsoma
from tiledbsoma.experimental.ingest import from_cxg_spatial_h5ad

In [2]:
# NOTE: Replace value of `my_h5ad_path` with an appropriate path to a cellxgene h5ad spatial file
my_h5ad_path = "../../../ps_stuff/spatial_test_datasets/c63d5cb4-1046-4948-a188-e6af50ef90f4.h5ad"

from_cxg_spatial_h5ad(
    input_h5ad_path=my_h5ad_path,
    experiment_uri="soma-spatial",
    measurement_name="RNA",
    scene_name="c63d5cb4-1046-4948-a188-e6af50ef90f4",
    uns_keys=[],
)

<frozen abc>:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.



'soma-spatial'

## Inspect the structure of the SOMA object on disk

In [3]:
sp = tiledbsoma.open("soma-spatial")
sp

<Experiment 'soma-spatial' (open for 'r') (3 items)
    'spatial': 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial' (unopened)
    'obs': 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/obs' (unopened)
    'ms': 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/ms' (unopened)>

In [4]:
sp.spatial["c63d5cb4-1046-4948-a188-e6af50ef90f4"]

<Scene 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial/c63d5cb4-1046-4948-a188-e6af50ef90f4' (open for 'r') (3 items)
    'img': 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial/c63d5cb4-1046-4948-a188-e6af50ef90f4/img' (unopened)
    'varl': 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial/c63d5cb4-1046-4948-a188-e6af50ef90f4/varl' (unopened)
    'obsl': 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial/c63d5cb4-1046-4948-a188-e6af50ef90f4/obsl' (unopened)>

In [5]:
sp.spatial["c63d5cb4-1046-4948-a188-e6af50ef90f4"]["img"]["hires"]

<DenseNDArray 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial/c63d5cb4-1046-4948-a188-e6af50ef90f4/img/hires' (open for 'r')>

In [6]:
hires_ndarray = sp.spatial["c63d5cb4-1046-4948-a188-e6af50ef90f4"]["img"]["hires"].read()
hires_ndarray.to_numpy()

array([[[237, 237, 237, ..., 237, 237, 237],
        [237, 237, 237, ..., 237, 237, 237],
        [237, 237, 237, ..., 237, 237, 237],
        ...,
        [237, 237, 237, ..., 234, 234, 233],
        [237, 237, 237, ..., 234, 234, 233],
        [237, 237, 237, ..., 234, 234, 233]],

       [[241, 241, 241, ..., 240, 240, 240],
        [241, 241, 241, ..., 240, 240, 240],
        [241, 241, 241, ..., 240, 240, 240],
        ...,
        [241, 241, 241, ..., 237, 237, 237],
        [241, 241, 241, ..., 237, 237, 236],
        [241, 241, 241, ..., 237, 237, 236]],

       [[240, 240, 240, ..., 243, 243, 243],
        [240, 240, 240, ..., 243, 243, 243],
        [240, 240, 240, ..., 243, 243, 243],
        ...,
        [240, 240, 240, ..., 240, 240, 239],
        [240, 240, 240, ..., 240, 240, 238],
        [240, 240, 240, ..., 240, 240, 238]]], dtype=uint8)

In [7]:
sp.spatial["c63d5cb4-1046-4948-a188-e6af50ef90f4"]["obsl"]

<DataFrame 'file:///Users/psridharan/code/cellxgene-census/tools/cellxgene_census_builder/spatial_dev_tools/soma-spatial/spatial/c63d5cb4-1046-4948-a188-e6af50ef90f4/obsl' (open for 'r')>

In [8]:
obsl_df = sp.spatial["c63d5cb4-1046-4948-a188-e6af50ef90f4"]["obsl"]
obsl_df.read().concat().to_pandas()

,y,x,soma_joinid,obs_id,in_tissue,array_row,array_col,sample,n_counts,log_counts,...,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid,_soma_geometry
0,269,207,475,ACGCCTGACACGCGCT-1,0,0,0,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,RtI>4TTINp,8.912657
1,269,254,351,ACAGGAGGCGCAGCCG-1,0,2,0,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,1NhO+d24#2,8.912657
2,269,350,897,AGTGGGAGTATACACG-1,0,6,0,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,)WcdK+K-I&,8.912657
3,269,779,1956,CGCAATCGATCATTAG-1,0,24,0,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,dXpvvuF)=I,8.912657
4,269,827,69,AAATGTATCTTATCCC-1,0,26,0,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,P&gHd-$`ro,8.912657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,2011,1328,4754,TTAGGATGGGAGGGTA-1,1,47,127,WS_PLA_S9101770,3707.0,8.217978,...,uterine smooth muscle cell,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,Qf1g!`$s)c,8.912657
4988,2011,1614,4178,TCCGGCTGTCGGGTCG-1,1,59,127,WS_PLA_S9101770,2860.0,7.958577,...,uterine smooth muscle cell,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,C<G$z5^2qT,8.912657
4989,2011,1853,4702,TTAACTGATCGTTTGG-1,0,69,127,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,fbCxa)n{M},8.912657
4990,2011,1900,4678,TGTTCTCATACTATAG-1,0,71,127,WS_PLA_S9101770,NaN,NaN,...,unknown,Visium Spatial Gene Expression,normal,Homo sapiens,unknown,decidua basalis,unknown,Carnegie stage 23,v+}g+r2P+{,8.912657
